# Day 11_3: 텍스트 마이닝 종합 프로젝트 - 정답 노트북

이 노트북은 실습 퀴즈의 정답과 풀이 설명을 포함합니다.

---

In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
import re
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

---

### Q1. 데이터 로드 및 기본 통계 ⭐

**문제**: `datasets/product_reviews.csv` 파일을 로드하고, 전체 리뷰 수, 고유 제품 수, 평균 평점을 출력하세요.

In [ ]:
# Q1 정답
df = pd.read_csv('datasets/product_reviews.csv')

total_reviews = len(df)
unique_products = df['product_name'].nunique()
avg_rating = df['rating'].mean()

print(f"전체 리뷰 수: {total_reviews}개")
print(f"고유 제품 수: {unique_products}개")
print(f"평균 평점: {avg_rating:.2f}점")

In [ ]:
# 테스트
assert total_reviews == 505, "리뷰 수 확인"
assert unique_products > 0, "제품 수 확인"
assert 1 <= avg_rating <= 5, "평점 범위 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: pandas의 기본 함수를 활용하여 데이터 요약 통계를 산출합니다.

**핵심 개념**:
- `len(df)`: DataFrame의 행 수
- `df['col'].nunique()`: 고유값 개수
- `df['col'].mean()`: 평균값

**실무 팁**: 데이터 로드 후 항상 `df.shape`, `df.info()`, `df.describe()`로 기본 탐색을 수행하세요.

---

### Q2. 텍스트 전처리 함수 구현 ⭐

**문제**: 텍스트에서 특수문자를 제거하고 소문자로 변환하는 함수를 구현하세요.

In [ ]:
# Q2 정답
def clean_text(text):
    """텍스트 전처리 함수"""
    # 소문자 변환
    text = text.lower()
    # 특수문자 제거 (한글, 영문, 숫자, 공백만 유지)
    text = re.sub(r'[^가-힣a-z0-9\s]', '', text)
    # 연속 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 테스트
sample = "화면이 정말 선명해요!! 최고입니다 :)"
result = clean_text(sample)
print(f"원본: {sample}")
print(f"전처리: {result}")

In [ ]:
# 테스트
assert '!' not in result, "특수문자 제거 확인"
assert ':' not in result, "이모티콘 제거 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: 정규표현식으로 불필요한 문자를 제거합니다.

**핵심 개념**:
- `re.sub(pattern, replacement, text)`: 패턴 매칭 후 치환
- `[^...]`: 대괄호 안 문자를 제외한 모든 것
- `가-힣`: 한글 음절 범위

**흔한 실수**:
- 한글 범위를 `ㄱ-ㅎ가-힣`로 쓰면 자음/모음도 포함됨
- `\s+`로 연속 공백 제거 안 하면 공백이 여러 개 남음

---

### Q3. 평점별 리뷰 수 계산 ⭐⭐

**문제**: 평점별 리뷰 수를 계산하고 Plotly 막대 그래프로 시각화하세요.

In [ ]:
# Q3 정답
rating_counts = df['rating'].value_counts().sort_index()

print("평점별 리뷰 수:")
for rating, count in rating_counts.items():
    print(f"  {rating}점: {count}개")

# 시각화
fig = px.bar(
    x=rating_counts.index,
    y=rating_counts.values,
    labels={'x': '평점', 'y': '리뷰 수'},
    title='평점별 리뷰 수 분포',
    color=rating_counts.values,
    color_continuous_scale='RdYlGn'
)
fig.update_layout(height=400, showlegend=False)
fig.show()

In [ ]:
# 테스트
assert len(rating_counts) == 5, "1-5점 모두 존재"
assert rating_counts.sum() == len(df), "총합 일치"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: `value_counts()`로 빈도 계산 후 `sort_index()`로 정렬합니다.

**핵심 개념**:
- `value_counts()`: 값별 빈도 계산
- `sort_index()`: 인덱스 기준 정렬
- `px.bar()`: Plotly Express 막대 그래프

**실무 팁**: 색상 스케일 `RdYlGn`(빨강-노랑-초록)은 평점 시각화에 적합합니다.

---

### Q4. 카테고리별 평균 평점 계산 ⭐⭐

**문제**: 카테고리별 평균 평점을 계산하고, 가장 높은/낮은 카테고리를 출력하세요.

In [ ]:
# Q4 정답
category_rating = df.groupby('category')['rating'].mean().sort_values(ascending=False)

best_category = category_rating.idxmax()
best_score = category_rating.max()

worst_category = category_rating.idxmin()
worst_score = category_rating.min()

print(f"최고 평점 카테고리: {best_category} ({best_score:.2f}점)")
print(f"최저 평점 카테고리: {worst_category} ({worst_score:.2f}점)")

In [ ]:
# 테스트
assert best_score >= worst_score, "최고 >= 최저"
assert 1 <= best_score <= 5, "평점 범위 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: `groupby().mean()`으로 그룹별 평균을 계산합니다.

**핵심 개념**:
- `groupby('col')['target'].mean()`: 그룹별 평균
- `idxmax()`, `idxmin()`: 최대/최소값의 인덱스

**대안 솔루션**:
```python
# nlargest/nsmallest 사용
top = category_rating.nlargest(1)
bottom = category_rating.nsmallest(1)
```

---

### Q5. 키워드 빈도 분석 ⭐⭐⭐

**문제**: 전체 리뷰에서 가장 많이 등장하는 단어 상위 10개를 추출하세요.

In [ ]:
# Q5 정답

# 텍스트 전처리
df['cleaned'] = df['review_text'].apply(lambda x: re.sub(r'[^가-힣a-z0-9\s]', ' ', x.lower()))

# 토큰화 (공백 기준, 2글자 이상)
all_words = []
for text in df['cleaned']:
    words = [w for w in text.split() if len(w) >= 2]
    all_words.extend(words)

# 빈도 계산
word_freq = Counter(all_words)
top_10 = word_freq.most_common(10)

print("상위 10개 단어:")
for word, count in top_10:
    print(f"  {word}: {count}회")

In [ ]:
# 테스트
assert len(top_10) == 10, "10개 추출 확인"
assert top_10[0][1] >= top_10[-1][1], "내림차순 정렬"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: 전처리 후 `Counter`로 빈도를 계산합니다.

**핵심 개념**:
- `Counter(list)`: 요소별 빈도 계산
- `.most_common(n)`: 상위 n개 반환
- 리스트 컴프리헨션으로 필터링

**흔한 실수**:
- 전처리 없이 토큰화하면 특수문자가 단어에 붙음
- 1글자 단어(조사 등)를 포함하면 의미 없는 결과

---

### Q6. 긍정/부정 리뷰 비교 ⭐⭐⭐

**문제**: 긍정 리뷰(4-5점)와 부정 리뷰(1-2점)의 평균 리뷰 길이를 비교하세요.

In [ ]:
# Q6 정답

# 리뷰 길이 계산
df['review_length'] = df['review_text'].str.len()

# 긍정/부정 필터링
positive_reviews = df[df['rating'] >= 4]
negative_reviews = df[df['rating'] <= 2]

pos_avg_length = positive_reviews['review_length'].mean()
neg_avg_length = negative_reviews['review_length'].mean()

print(f"긍정 리뷰 평균 길이: {pos_avg_length:.0f}자")
print(f"부정 리뷰 평균 길이: {neg_avg_length:.0f}자")
print(f"\n차이: {abs(pos_avg_length - neg_avg_length):.0f}자")

In [ ]:
# 테스트
assert pos_avg_length > 0, "긍정 리뷰 길이 확인"
assert neg_avg_length > 0, "부정 리뷰 길이 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: 조건 필터링 후 문자열 길이의 평균을 계산합니다.

**핵심 개념**:
- `df['col'].str.len()`: 문자열 길이
- 불리언 인덱싱: `df[df['col'] >= value]`

**인사이트**: 일반적으로 부정 리뷰가 더 길 수 있습니다 (불만 상세 설명 경향)

---

### Q7. TF-IDF로 중요 키워드 추출 ⭐⭐⭐

**문제**: TF-IDF 점수 상위 10개 키워드를 추출하세요.

In [ ]:
# Q7 정답

# 토큰화된 텍스트 준비
df['tokens_str'] = df['cleaned'].apply(lambda x: ' '.join([w for w in x.split() if len(w) >= 2]))

# TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=100)
tfidf_matrix = tfidf.fit_transform(df['tokens_str'])

# 상위 키워드 추출
feature_names = tfidf.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1
tfidf_ranking = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)

print("TF-IDF 상위 10개 키워드:")
for word, score in tfidf_ranking[:10]:
    print(f"  {word}: {score:.2f}")

In [ ]:
# 테스트
assert len(tfidf_ranking) >= 10, "키워드 10개 이상"
assert tfidf_ranking[0][1] >= tfidf_ranking[9][1], "내림차순 정렬"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: sklearn의 TfidfVectorizer로 TF-IDF 행렬을 생성하고 열별 합계로 중요도를 계산합니다.

**핵심 개념**:
- TF-IDF = Term Frequency * Inverse Document Frequency
- 자주 등장하지만 특정 문서에만 나오는 단어가 높은 점수
- `.A1`: sparse matrix를 1D array로 변환

**TF-IDF vs 단순 빈도**:
- 빈도: 자주 등장하는 단어 (일반적인 단어 포함)
- TF-IDF: 문서를 구별하는 중요한 단어

---

### Q8. 감성 분석 함수 구현 ⭐⭐⭐⭐

**문제**: 긍정/부정 단어 사전 기반 감성 분석 함수를 구현하세요.

In [ ]:
# Q8 정답

# 감성 사전
positive_words = {'좋아요', '최고', '만족', '훌륭', '추천', '편해요', '좋습니다', '뛰어나', '깔끔', '빠르'}
negative_words = {'별로', '실망', '불편', '아쉬워', '비싸', '느려', '무거워', '불안', '부족', '아쉽'}

def analyze_sentiment(text):
    """감성 분석 함수"""
    pos_count = sum(1 for word in positive_words if word in text)
    neg_count = sum(1 for word in negative_words if word in text)
    
    if pos_count > neg_count:
        return '긍정'
    elif neg_count > pos_count:
        return '부정'
    else:
        return '중립'

# 전체 리뷰에 적용
df['sentiment'] = df['review_text'].apply(analyze_sentiment)

# 결과 출력
sentiment_dist = df['sentiment'].value_counts()
print("감성 분포:")
for sentiment, count in sentiment_dist.items():
    print(f"  {sentiment}: {count}개 ({count/len(df)*100:.1f}%)")

In [ ]:
# 테스트
assert set(df['sentiment'].unique()).issubset({'긍정', '부정', '중립'}), "감성 라벨 확인"
assert sentiment_dist.sum() == len(df), "총합 일치"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: 사전 기반으로 긍정/부정 단어 출현 횟수를 비교합니다.

**핵심 개념**:
- 사전 기반 감성 분석: 규칙 기반, 해석 용이
- 집합 연산으로 효율적인 단어 매칭

**대안 솔루션**:
```python
# 감성 점수 (-1 ~ 1) 반환
def sentiment_score(text):
    pos = sum(1 for w in positive_words if w in text)
    neg = sum(1 for w in negative_words if w in text)
    total = pos + neg
    return (pos - neg) / total if total > 0 else 0
```

---

### Q9. 제품별 리뷰 분석 대시보드 ⭐⭐⭐⭐

**문제**: 특정 제품의 리뷰를 분석하는 대시보드를 만드세요.

In [ ]:
# Q9 정답

# 제품 필터링
product_name = '삼성 갤럭시 S24'
product_df = df[df['product_name'] == product_name].copy()

print(f"제품: {product_name}")
print(f"리뷰 수: {len(product_df)}개")
print(f"평균 평점: {product_df['rating'].mean():.2f}점")

# 대시보드 생성
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('평점 분포', '월별 리뷰 수'),
    specs=[[{"type": "pie"}, {"type": "bar"}]]
)

# 1) 평점 분포 파이 차트
rating_dist = product_df['rating'].value_counts().sort_index()
fig.add_trace(
    go.Pie(labels=rating_dist.index.astype(str) + '점', values=rating_dist.values, name='평점'),
    row=1, col=1
)

# 2) 월별 리뷰 수
product_df['month'] = pd.to_datetime(product_df['review_date']).dt.to_period('M').astype(str)
monthly = product_df.groupby('month').size()
fig.add_trace(
    go.Bar(x=monthly.index, y=monthly.values, name='월별'),
    row=1, col=2
)

fig.update_layout(height=400, title_text=f'{product_name} 리뷰 분석', showlegend=False)
fig.show()

In [ ]:
# 테스트
assert len(product_df) > 0, "제품 데이터 존재"
assert 'month' in product_df.columns, "월 컬럼 생성"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: `make_subplots()`로 여러 차트를 하나의 대시보드에 배치합니다.

**핵심 개념**:
- `make_subplots()`: 서브플롯 레이아웃 정의
- `specs`: 각 서브플롯의 차트 유형 지정
- `add_trace(row, col)`: 특정 위치에 차트 추가

**실무 팁**: 대시보드는 핵심 지표를 한눈에 볼 수 있도록 구성하세요.

---

### Q10. 종합 인사이트 리포트 생성 ⭐⭐⭐⭐⭐

**문제**: 전체 데이터를 분석하여 종합 리포트를 생성하세요.

In [ ]:
# Q10 정답

print("="*60)
print("          전자제품 리뷰 분석 종합 리포트")
print("="*60)

# 1. 전체 데이터 요약
print("\n[1] 데이터 요약")
print(f"    - 분석 기간: {df['review_date'].min()} ~ {df['review_date'].max()}")
print(f"    - 총 리뷰 수: {len(df):,}개")
print(f"    - 분석 제품: {df['product_name'].nunique()}개")
print(f"    - 평균 평점: {df['rating'].mean():.2f}점")
print(f"    - 긍정 리뷰 비율: {(df['rating'] >= 4).mean()*100:.1f}%")

# 2. 카테고리별 분석 (리뷰 10개 이상)
print("\n[2] 카테고리별 만족도")
cat_stats = df.groupby('category').agg({
    'rating': ['mean', 'count']
}).round(2)
cat_stats.columns = ['평균평점', '리뷰수']
cat_stats = cat_stats[cat_stats['리뷰수'] >= 10].sort_values('평균평점', ascending=False)

best_cat = cat_stats.index[0]
worst_cat = cat_stats.index[-1]
print(f"    - 최고 만족도: {best_cat} ({cat_stats.loc[best_cat, '평균평점']}점)")
print(f"    - 최저 만족도: {worst_cat} ({cat_stats.loc[worst_cat, '평균평점']}점)")

# 3. 긍정/부정 키워드
print("\n[3] 주요 키워드")
positive_df = df[df['rating'] >= 4]
negative_df = df[df['rating'] <= 2]

pos_words = Counter(' '.join(positive_df['cleaned']).split()).most_common(10)
neg_words = Counter(' '.join(negative_df['cleaned']).split()).most_common(10)

print(f"    긍정 리뷰 키워드: {[w for w, c in pos_words[:5]]}")
print(f"    부정 리뷰 키워드: {[w for w, c in neg_words[:5]]}")

# 4. 비즈니스 권장사항
print("\n[4] 비즈니스 권장사항")
print("    1. 가격 경쟁력 강화")
print("       - '비싸다' 키워드가 부정 리뷰에 빈번하게 등장")
print("       - 가격 대비 가치 인식 개선 필요")
print("")
print("    2. 제품 품질 개선")
print("       - 배터리, 발열 관련 불만 다수")
print("       - 하드웨어 품질 검수 강화 권장")
print("")
print("    3. AS 서비스 강화")
print("       - '고장', 'AS' 관련 부정 리뷰 존재")
print("       - 사후 서비스 접근성 개선 필요")

print("\n" + "="*60)
print("                  리포트 작성 완료")
print("="*60)

In [ ]:
# 테스트
assert len(cat_stats) > 0, "카테고리 분석 완료"
assert len(pos_words) >= 5, "긍정 키워드 추출"
assert len(neg_words) >= 5, "부정 키워드 추출"
print("테스트 통과!")

### 풀이 설명

**접근 방법**: 분석 결과를 체계적인 리포트 형식으로 구성합니다.

**핵심 개념**:
- 데이터 요약: 기본 통계로 전체 그림 제시
- 세그먼트 분석: 카테고리/제품별 비교
- 키워드 분석: 긍정/부정 요인 파악
- 권장사항: 데이터 기반 행동 제안

**리포트 구성 팁**:
1. **What**: 데이터가 무엇을 보여주는가
2. **So What**: 그래서 무엇이 중요한가
3. **Now What**: 어떤 조치가 필요한가

**실무 팁**: 경영진 보고 시 숫자와 함께 구체적인 행동 제안을 포함하면 실행력이 높아집니다.

---

## 학습 정리

### 텍스트 마이닝 종합 프로젝트 핵심

| 단계 | 주요 작업 | 핵심 기법 |
|------|----------|----------|
| 1. 데이터 로드 | CSV 읽기, EDA | pandas, describe() |
| 2. 전처리 | 정규화, 토큰화 | regex, split() |
| 3. 키워드 분석 | 빈도, TF-IDF | Counter, TfidfVectorizer |
| 4. 감성 분석 | 사전 기반 분류 | 커스텀 함수 |
| 5. 시각화 | 대시보드 구축 | Plotly, make_subplots |
| 6. 인사이트 | 권장사항 도출 | 비즈니스 해석 |

### 프로젝트 완료!

이 패턴을 다른 텍스트 데이터(뉴스, SNS, 고객 상담 등)에도 적용해보세요.